In [1]:
import sympy
import numpy as np
import pandas as pd
from sympy import *
from sympy.stats import *
from scipy.integrate import odeint 

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

In [4]:
init_printing( use_latex='mathjax' )

## 环境配置

### 函数定义

In [10]:
##定义函数
#计算R_0,R_1,R_2
def Rfunc(n,λl):
    if n==0:
        return eye(A.cols)
    else:
        return (A-λl[n-1]*eye(A.cols))*Rfunc(n-1,λl)
    
#计算r(t)
def rfunc(n,λl):
    if n==1:
        return exp(λl[n-1]*t)
    else:
        return exp(λl[n-1]*t)*integrate(exp(-λl[n-1]*t)*rfunc(n-1,λl),(t,0,t))
# 计算exp(A*t)    
def Putzer(A):
    λl = []
    n = A.cols
    for valst in A.eigenvects():
        for i in range(valst[1]):
            λl.append(valst[0])
    λl.sort(key = lambda x:re(x))
    suma = zeros(n,n)
    for i in range(1,n+1):
        suma += rfunc(i,λl)*Rfunc(i-1,λl)
    return display(A.eigenvects()),display(λl),display([simplify(Rfunc(i,λl)) for i in range(n)]),display([expand(rfunc(i,λl),complex=True).simplify() for i in range(1,n+1)]),display(simplify(suma).expand(complex=True).simplify())

In [11]:
# 二阶线性非齐次常系数微分系统
def SONHLS(p,q,f,t0):
    sol = solve(x**2+p*x+q)
    if p**2-4*q > 0:
        phi1 = exp(sol[0]*t)
        phi2 = exp(sol[1]*t)
    elif p**2-4*q == 0:
        phi1 = exp(sol[0]*t)
        phi2 = t*exp(sol[0]*t)
    else:
        phi1 = exp(re(sol[0])*t)*cos(abs(im(sol[0]))*t)
        phi2 = exp(re(sol[0]*t))*sin(abs(im(sol[0]))*t)
    X = Matrix([[phi1,phi2],[diff(phi1),diff(phi2)]])
    dv1 = simplify((X**(-1)*Matrix([[0],[f]]))[0])
    dv2 = simplify((X**(-1)*Matrix([[0],[f]]))[1])
    v1 = integrate(dv1,(t,t0,t))
    v2 = integrate(dv2,(t,t0,t))
    return display([phi1,phi2]),display([dv1,dv2]),display([v1,v2])

In [12]:
# 幂级数法求变系数微分方程
def AnalyticPower(a_1,a_0,b,nmax,need):
    c0 = symbols('c0')
    clst = symbols('c_0:{}'.format(nmax))
    x0tlst = []
    for k in range(0,nneed):
        x0tlst.append(clst[k]*t**k) 
    x1tlst = [diff(x0tlst[i],(t,1)) for i in range(1,nneed)]
    x2tlst = [diff(x0tlst[i],(t,2)) for i in range(2,nneed)]
    a0lst = []
    for k in range(0,nneed-2):
        a0lst.append(diff(a_0,(t,k)).subs(t,0)*t**k/factorial(k)) 
    a1lst = []
    for k in range(0,nneed-2):
        a1lst.append(diff(a_1,(t,k)).subs(t,0)*t**k/factorial(k)) 
    blst = []
    for k in range(0,nneed-2):
        blst.append(diff(b,(t,k)).subs(t,0)*t**k/factorial(k)) 
    sl1 = []
    for n in range(0,nneed-2):
        lin = 0
        for k in range(0,n+1):
            lin += a1lst[k]*x1tlst[n-k]
        sl1.append(lin)
    sl0 = []
    for n in range(0,nneed-2):
        lin = 0
        for k in range(0,n+1):
            lin += simplify(a0lst[k]*x0tlst[n-k])
        sl0.append(lin)
    tlst = []
    for n in range(0,8):
        tlst.append(simplify(x2tlst[n]+sl1[n]+sl0[n]-blst[n])/t**n)
    return display(tlst[0]),display(tlst[1]),display(tlst[2]),display(tlst[3]),display(tlst[4]),display(tlst[5]),display(tlst[6])

In [13]:
# Hurwitz矩阵
def Hurwitz(A):
    P = det(A-λ*eye(A.cols))
    n = A.cols
    alst = Poly(P).coeffs()
    if alst[0] < 0:
        alst = [-val for val in alst]
    H = eye(n)
    for i in range(n):
        for j in range(n):
            if i >= 2*j-n+1 and i <= 2*j+1:
                H[i,j] = alst[2*j-i+1]
            else:
                H[i,j] = 0
    blst = list(map(lambda x:re(x).evalf(),solve(det(A-λ*eye(A.cols)))))
    return display(alst),display(H),display(blst)

### 变量定义

In [14]:
x,y,z,m,n,a,λ,u,v,a,b,c,d,t,i,k = symbols('x y z m n a λ u v a b c d t i k')
t = symbols('t', real=True)
λ1,λ2,λ3 = symbols('λ1 λ2 λ3') 

## 求基解矩阵

Putzer法求Exp(At)

$$
r_1(t) = e^{\lambda_1 t}, \ R_0 = I_n \\
r_j(t) = e^{\lambda_j t}\int_0^t e^{-\lambda_j s}r_{j-1}(s) ds, \ R_{j-1} = \prod_{k=1}^{j-1}(A-\lambda_k I_n)
$$

$$
e^{At} = \sum_{j=1}^{n} r_j(t)R_{j-1}
$$

In [17]:
# 运行这块
A = Matrix([[-1,-2],[3,4]])
Putzer(A) 
exp(A*t)

⎡⎛      ⎡⎡-1⎤⎤⎞  ⎛      ⎡⎡-2/3⎤⎤⎞⎤
⎢⎜1, 1, ⎢⎢  ⎥⎥⎟, ⎜2, 1, ⎢⎢    ⎥⎥⎟⎥
⎣⎝      ⎣⎣1 ⎦⎦⎠  ⎝      ⎣⎣ 1  ⎦⎦⎠⎦

[1, 2]

⎡⎡1  0⎤  ⎡-2  -2⎤⎤
⎢⎢    ⎥, ⎢      ⎥⎥
⎣⎣0  1⎦  ⎣3   3 ⎦⎦

⎡ t  ⎛ t    ⎞  t⎤
⎣ℯ , ⎝ℯ  - 1⎠⋅ℯ ⎦

⎡⎛       t⎞  t    ⎛     t⎞  t⎤
⎢⎝3 - 2⋅ℯ ⎠⋅ℯ   2⋅⎝1 - ℯ ⎠⋅ℯ ⎥
⎢                            ⎥
⎢  ⎛ t    ⎞  t  ⎛   t    ⎞  t⎥
⎣3⋅⎝ℯ  - 1⎠⋅ℯ   ⎝3⋅ℯ  - 2⎠⋅ℯ ⎦

(None, None, None, None, None)

⎡     2⋅t      t       2⋅t      t⎤
⎢- 2⋅ℯ    + 3⋅ℯ   - 2⋅ℯ    + 2⋅ℯ ⎥
⎢                                ⎥
⎢    2⋅t      t       2⋅t      t ⎥
⎣ 3⋅ℯ    - 3⋅ℯ     3⋅ℯ    - 2⋅ℯ  ⎦

In [18]:
#输入系数矩阵
A = Matrix([[3,-1,1],[2,0,1],[1,-1,2]])
A
#step1 特征多项式
(A-λ*eye(A.cols)).det().factor()
λl = []
for valst in A.eigenvects():
    for i in range(valst[1]):
        λl.append(valst[0])
λl

⎡3  -1  1⎤
⎢        ⎥
⎢2  0   1⎥
⎢        ⎥
⎣1  -1  2⎦

        2        
-(λ - 2) ⋅(λ - 1)

[1, 2, 2]

In [19]:
for i in range(0,3):
    simplify(Rfunc(i,λl))

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

⎡2  -1  1⎤
⎢        ⎥
⎢2  -1  1⎥
⎢        ⎥
⎣1  -1  1⎦

⎡1  -1  1⎤
⎢        ⎥
⎢1  -1  1⎥
⎢        ⎥
⎣0  0   0⎦

In [14]:
for i in range(1,4):
    expand(rfunc(i,λl),complex=True).simplify()

 t
ℯ 

⎛ t    ⎞  t
⎝ℯ  - 1⎠⋅ℯ 

⎛   t    t    ⎞  t
⎝t⋅ℯ  - ℯ  + 1⎠⋅ℯ 

In [20]:
suma = zeros(3,3)
for i in range(1,4):
    suma += rfunc(i,λl)*Rfunc(i-1,λl)

In [21]:
simplify(suma).expand(complex=True).simplify()

⎡            2⋅t             2⋅t         2⋅t⎤
⎢   (t + 1)⋅ℯ            -t⋅ℯ         t⋅ℯ   ⎥
⎢                                           ⎥
⎢⎛   t    t    ⎞  t  ⎛     t    ⎞  t     2⋅t⎥
⎢⎝t⋅ℯ  + ℯ  - 1⎠⋅ℯ   ⎝- t⋅ℯ  + 1⎠⋅ℯ   t⋅ℯ   ⎥
⎢                                           ⎥
⎢   ⎛ t    ⎞  t        ⎛     t⎞  t      2⋅t ⎥
⎣   ⎝ℯ  - 1⎠⋅ℯ         ⎝1 - ℯ ⎠⋅ℯ      ℯ    ⎦

In [17]:
#验证

In [22]:
exp(A*t).expand(complex=True)

⎡     2⋅t    2⋅t            2⋅t        2⋅t⎤
⎢  t⋅ℯ    + ℯ           -t⋅ℯ        t⋅ℯ   ⎥
⎢                                         ⎥
⎢   2⋅t    2⋅t    t       2⋅t    t     2⋅t⎥
⎢t⋅ℯ    + ℯ    - ℯ   - t⋅ℯ    + ℯ   t⋅ℯ   ⎥
⎢                                         ⎥
⎢     2⋅t    t           2⋅t    t     2⋅t ⎥
⎣    ℯ    - ℯ         - ℯ    + ℯ     ℯ    ⎦

In [19]:
A.jordan_form()

⎛⎡0  1  0⎤  ⎡1  0  0⎤⎞
⎜⎢       ⎥  ⎢       ⎥⎟
⎜⎢1  1  0⎥, ⎢0  2  1⎥⎟
⎜⎢       ⎥  ⎢       ⎥⎟
⎝⎣1  0  1⎦  ⎣0  0  2⎦⎠

In [20]:
A.eigenvects()

⎡⎛      ⎡⎡0⎤⎤⎞  ⎛      ⎡⎡1⎤⎤⎞⎤
⎢⎜      ⎢⎢ ⎥⎥⎟  ⎜      ⎢⎢ ⎥⎥⎟⎥
⎢⎜1, 1, ⎢⎢1⎥⎥⎟, ⎜2, 2, ⎢⎢1⎥⎥⎟⎥
⎢⎜      ⎢⎢ ⎥⎥⎟  ⎜      ⎢⎢ ⎥⎥⎟⎥
⎣⎝      ⎣⎣1⎦⎦⎠  ⎝      ⎣⎣0⎦⎦⎠⎦

## 幂级数法求线性非齐次微分方程

二阶非齐次线性微分方程
$$
x^{''}(t)+a_1(t)x^{'}(t)+a_0(t)x(t)=b(t)
$$  
$a_1(t)$, $a_0(t)$, $b(t)$在0处解析

In [23]:
#运行
#a_1,a_0,b的定义
a_1 = 1/(1-t)
a_0 = sin(t)
b = 0
nmax = 20 #不要改
nneed = 12 #不要改

In [24]:
#运行
AnalyticPower(a_1,a_0,b,nmax,nneed)

c₁ + 2⋅c₂

c₀ + c₁ + 2⋅c₂ + 6⋅c₃

2⋅c₁ + 2⋅c₂ + 3⋅c₃ + 12⋅c₄

  c₀                                  
- ── + c₁ + 3⋅c₂ + 3⋅c₃ + 4⋅c₄ + 20⋅c₅
  6                                   

5⋅c₁                                    
──── + 2⋅c₂ + 4⋅c₃ + 4⋅c₄ + 5⋅c₅ + 30⋅c₆
 6                                      

 c₀        11⋅c₂                                    
─── + c₁ + ───── + 3⋅c₃ + 5⋅c₄ + 5⋅c₅ + 6⋅c₆ + 42⋅c₇
120          6                                      

121⋅c₁          17⋅c₃                                    
────── + 2⋅c₂ + ───── + 4⋅c₄ + 6⋅c₅ + 6⋅c₆ + 7⋅c₇ + 56⋅c₈
 120              6                                      

(None, None, None, None, None, None, None)

In [23]:
c0 = symbols('c0')
clst = symbols('c_0:{}'.format(nmax))
clst

(c₀, c₁, c₂, c₃, c₄, c₅, c₆, c₇, c₈, c₉, c₁₀, c₁₁, c₁₂, c₁₃, c₁₄, c₁₅, c₁₆, c₁
₇, c₁₈, c₁₉)

In [24]:
x0tlst = []
for k in range(0,nneed):
    x0tlst.append(clst[k]*t**k) 
x0tlst
x1tlst = [diff(x0tlst[i],(t,1)) for i in range(1,nneed)]
x1tlst
x2tlst = [diff(x0tlst[i],(t,2)) for i in range(2,nneed)]
x2tlst

⎡              2      3      4      5      6      7      8      9       10    
⎣c₀, c₁⋅t, c₂⋅t , c₃⋅t , c₄⋅t , c₅⋅t , c₆⋅t , c₇⋅t , c₈⋅t , c₉⋅t , c₁₀⋅t  , c₁

   11⎤
₁⋅t  ⎦

⎡                  2        3        4        5        6        7        8    
⎣c₁, 2⋅c₂⋅t, 3⋅c₃⋅t , 4⋅c₄⋅t , 5⋅c₅⋅t , 6⋅c₆⋅t , 7⋅c₇⋅t , 8⋅c₈⋅t , 9⋅c₉⋅t , 10

      9          10⎤
⋅c₁₀⋅t , 11⋅c₁₁⋅t  ⎦

⎡                     2         3         4         5         6         7     
⎣2⋅c₂, 6⋅c₃⋅t, 12⋅c₄⋅t , 20⋅c₅⋅t , 30⋅c₆⋅t , 42⋅c₇⋅t , 56⋅c₈⋅t , 72⋅c₉⋅t , 90⋅

     8           9⎤
c₁₀⋅t , 110⋅c₁₁⋅t ⎦

In [25]:
a0lst = []
for k in range(0,nneed-2):
    a0lst.append(diff(a_0,(t,k)).subs(t,0)*t**k/factorial(k)) 
a0lst
a1lst = []
for k in range(0,nneed-2):
    a1lst.append(diff(a_1,(t,k)).subs(t,0)*t**k/factorial(k)) 
a1lst
blst = []
for k in range(0,nneed-2):
    blst.append(diff(b,(t,k)).subs(t,0)*t**k/factorial(k)) 
blst

⎡           3        5       7         9  ⎤
⎢         -t        t      -t         t   ⎥
⎢0, t, 0, ────, 0, ───, 0, ────, 0, ──────⎥
⎣          6       120     5040     362880⎦

⎡       2   3   4   5   6   7   8   9⎤
⎣1, t, t , t , t , t , t , t , t , t ⎦

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [26]:
sl1 = []
for n in range(0,nneed-2):
    lin = 0
    for k in range(0,n+1):
        lin += a1lst[k]*x1tlst[n-k]
    sl1.append(lin)
sl1

⎡                       2         2         2      3         3         3      
⎣c₁, c₁⋅t + 2⋅c₂⋅t, c₁⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t , c₁⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t  + 4⋅c

   3      4         4         4         4         4      5         5         5
₄⋅t , c₁⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t  + 4⋅c₄⋅t  + 5⋅c₅⋅t , c₁⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t 

         5         5         5      6         6         6         6         6 
 + 4⋅c₄⋅t  + 5⋅c₅⋅t  + 6⋅c₆⋅t , c₁⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t  + 4⋅c₄⋅t  + 5⋅c₅⋅t  

        6         6      7         7         7         7         7         7  
+ 6⋅c₆⋅t  + 7⋅c₇⋅t , c₁⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t  + 4⋅c₄⋅t  + 5⋅c₅⋅t  + 6⋅c₆⋅t  +

       7         7      8         8         8         8         8         8   
 7⋅c₇⋅t  + 8⋅c₈⋅t , c₁⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t  + 4⋅c₄⋅t  + 5⋅c₅⋅t  + 6⋅c₆⋅t  + 

      8         8         8      9           9         9         9         9  
7⋅c₇⋅t  + 8⋅c₈⋅t  + 9⋅c₉⋅t , c₁⋅t  + 10⋅c₁₀⋅t  + 2⋅c₂⋅t  + 3⋅c₃⋅t  + 4⋅c₄⋅t  +

       9         9         9         9        

In [27]:
sl0 = []
for n in range(0,nneed-2):
    lin = 0
    for k in range(0,n+1):
        lin += simplify(a0lst[k]*x0tlst[n-k])
    sl0.append(lin)
sl0

⎡                      3                4              5       5              
⎢             2    c₀⋅t        3    c₁⋅t        4  c₀⋅t    c₂⋅t        5  c₁⋅t
⎢0, c₀⋅t, c₁⋅t , - ───── + c₂⋅t , - ───── + c₃⋅t , ───── - ───── + c₄⋅t , ────
⎣                    6                6             120      6             120

6       6                7       7       7                8       8       8   
    c₃⋅t        6    c₀⋅t    c₂⋅t    c₄⋅t        7    c₁⋅t    c₃⋅t    c₅⋅t    
─ - ───── + c₅⋅t , - ───── + ───── - ───── + c₆⋅t , - ───── + ───── - ───── + 
      6               5040    120      6               5040    120      6     

           9        9       9       9        ⎤
    8  c₀⋅t     c₂⋅t    c₄⋅t    c₆⋅t        9⎥
c₇⋅t , ────── - ───── + ───── - ───── + c₈⋅t ⎥
       362880    5040    120      6          ⎦

In [28]:
tlst = []
for n in range(0,8):
    tlst.append(simplify(x2tlst[n]+sl1[n]+sl0[n]-blst[n])/t**n)
tlst[0]
tlst[1]
tlst[2]
tlst[3]
tlst[4]
tlst[5]
tlst[6]

c₁ + 2⋅c₂

c₀ + c₁ + 2⋅c₂ + 6⋅c₃

2⋅c₁ + 2⋅c₂ + 3⋅c₃ + 12⋅c₄

  c₀                                  
- ── + c₁ + 3⋅c₂ + 3⋅c₃ + 4⋅c₄ + 20⋅c₅
  6                                   

5⋅c₁                                    
──── + 2⋅c₂ + 4⋅c₃ + 4⋅c₄ + 5⋅c₅ + 30⋅c₆
 6                                      

 c₀        11⋅c₂                                    
─── + c₁ + ───── + 3⋅c₃ + 5⋅c₄ + 5⋅c₅ + 6⋅c₆ + 42⋅c₇
120          6                                      

121⋅c₁          17⋅c₃                                    
────── + 2⋅c₂ + ───── + 4⋅c₄ + 6⋅c₅ + 6⋅c₆ + 7⋅c₇ + 56⋅c₈
 120              6                                      

## 二阶非齐次线性微分方程

$$
x^{''}(t)+px^{'}(t)+qx(t)=f(t)
$$  

$$\phi_1(t)=e^{\lambda_1t},\  \phi_2(t)=e^{\lambda_2t}$$

$$\phi_1(t)=e^{\lambda t}, \ \phi_2(t)=te^{\lambda t}$$

$$\phi_1(t)=e^{at}cos(b), \ \phi_2(t)=e^{a t}sin(b)$$

基解矩阵
\begin{align}
X(t)=  
\begin{bmatrix}
\phi_1(t)  &\phi_2(t) \\
\phi_1(t)' &\phi_2(t)'
\end{bmatrix}
\end{align}

通解的形式为

$$
\vec x(t) = X(t)\left(X^{-1}(t_0)\vec x(t_0)+\int_{t_0}^t X^{-1}(s)\vec b(s) \ ds \right)
$$

In [29]:
#运行
x,y,z,m,n,a,λ,u,v,a,b,c,d,t,i,k = symbols('x y z m n a λ u v a b c d t i k')
t = symbols('t',real=True)
c1,c2 = symbols('c1 c2')

In [30]:
#运行
p = -2
q = 5
t0 = 1
f = exp(t)*cos(2*t)
SONHLS(p,q,f,t0)

⎡ t            t         ⎤
⎣ℯ ⋅cos(2⋅t), ℯ ⋅sin(2⋅t)⎦

⎡               2     ⎤
⎢-sin(4⋅t)   cos (2⋅t)⎥
⎢──────────, ─────────⎥
⎣    4           2    ⎦

⎡cos(4⋅t)   cos(4)  t   sin(2⋅t)⋅cos(2⋅t)   1   sin(2)⋅cos(2)⎤
⎢──────── - ──────, ─ + ───────────────── - ─ - ─────────────⎥
⎣   16        16    4           8           4         8      ⎦

(None, None, None)

In [31]:
# 验证
x = symbols('x', cls = Function)
diffeq = Eq(x(t).diff(t,2) +2*x(t).diff(t) + 1*x(t)-sin(t),f)
dsolve(diffeq, x(t))

                          t                  
                    -t   ℯ ⋅sin(2⋅t)   cos(t)
x(t) = (C₁ + C₂⋅t)⋅ℯ   + ─────────── - ──────
                              8          2   

## 线性系统的稳定性判断

### 线性齐次系统

线性系统的稳定性与基解矩阵挂钩，考虑f(t,s)=$\Vert X(t)X^{-1}(s) \Vert$, $t \geq s$
- (H)稳定 iff $f(t,s)$能被$K(s)$控制住
- (H)一致稳定 iff $f(t,s)$能被$K$控制住
- (H)渐进稳定 iff $f(t,s)$$\to$0，当固定s时
- (H)一致渐进稳定 iff $f(t,s)$一致收敛于与0，此时f(t,s)有一个指数阶的衰减速度

#### 线性齐次常系数系统

不难发现线性齐次系统是自治的，因此稳定与一致稳定，渐进与一致渐进等价

- 稳定 iff A的特征值实部非正，而且为0的部分全部落入简单型内   
- 渐进稳定 iff A的特征值实部全部为负
    - 如果根据A的特征多项式生成的Hurwitz Matrix正定，则所有特征值实部为负

##### Hurwitz Matrix

In [26]:
# 运行
A = Matrix([[-2,1,1],[1,-1,0],[1,1,-1]])
Hurwitz(A)

[1, 4, 3, -1]

⎡4  -1  0 ⎤
⎢         ⎥
⎢1  3   0 ⎥
⎢         ⎥
⎣0  4   -1⎦

[-1.44504186791263, -2.80193773580484, 0.246979603717467]

(None, None, None)

## 非线性系统稳定性判断

### 平面系统

\begin{align}
  x' &= ax+by \\
  y' &= cx+dy 
\end{align}
<hr>

值得注意的是线性变换不改变稳定性<br/>
计算矩阵A的特征值，分为3类情况

<hr>
1. (a): $\lambda_1,\lambda_2>0$, (b): $\lambda_1,\lambda_2<0$, (c): $\lambda_1<, \lambda_2>0$ 
<div align=center>
<img src="https://raw.githubusercontent.com/nymath/Figure/main/Differential_Equation/plannar_delta_geq.png"  width = "350" height = "290">
<center><font size=2>Figure 1 (a) unstable node, (b) stable node, (c) saddle point</font></center>
</div>
<hr>
2. (a): $\alpha > 0$, (b): $\alpha < 0$, (c): $\alpha = 0$ 
<div align=center>
<img src="https://raw.githubusercontent.com/nymath/Figure/main/Differential_Equation/plannar_delta_leq.png"  width = "350" height = "290">
<center><font size=2>Figure 2 (a) unstable focus, (b) stable focus, (c) center</font></center>
</div>
<hr>
3. (a): $Ker(A-\lambda I)=2$, (b):$Ker(A-\lambda I)=1$ 
<div align=center>
<img src="https://raw.githubusercontent.com/nymath/Figure/main/Differential_Equation/plannar_delta_eq.png"  width = "350" height = "290">
<center><font size=2>Figure 3 (a) unstable node, (b) unstable degenerate node</font></center>
</div>

### 自治系统

#### 线性化

记D为在均衡点出的Jocobian矩阵，我们给出两个充分条件
- 如果D的特征值实部为负，则渐进稳定
- 如果存在正实部，则不稳定

In [47]:
x1,x2,x3 = symbols('x1 x2 x3')

In [54]:
xlst = list(symbols('x_1:{}'.format(3)))

[x₁, x₂]

In [79]:
def linearization_test_stability(f):
    n = f.rows
    xlst = list(symbols('x1:{}'.format(n+1)))
    solve(f)
    display(f.jacobian(xlst).subs('x1',0).subs('x2'))

In [80]:
f = Matrix([x1*(1-x1-x2),1/4*x2*(2-3*x1-3*x2)])
linearization_test_stability(f)
solve(f)

⎡ 1 - x₂        0      ⎤
⎢                      ⎥
⎣-0.75⋅x₂  0.5 - 1.5⋅x₂⎦

[{x₁: 0.0, x₂: 0.0}, {x₁: 0.0, x₂: 0.666666666666667}, {x₁: 1.0, x₂: 0.0}]

#### Lyapunov第二法

我们选择Lyapunov函数$V(x_1,x_2)>0$ when $(x_1,x_2)\neq0$, define
$$
\dot{V}(x_1,x_2) = \langle \nabla V,\dot{x}\rangle
$$
<hr>
稳定性基本定理, if <br/>
- $V(x_1,x_2)$ is negative definite, then (0,0) is asypmpotically stable <br/> 
- $V(x_1,x_2)$ is equal to zero, then (0,0) is stable   <br/>
- $V(x_1,x_2)$ is positive definite, then (0,0) is unstable  <br/> 
<hr>
减弱的版本: <br/>
1. if $V(x_1,x_2)$ is positive, and the basin of $\dot{V}(x_1,x_2)$ does not contain the trajectory of system, then (0,0) is asympotically stable. <br/>
2. 如果$V(x_1,x_2)$在(0,0)的领域内是变号函数，但$\dot{V}(x_1,x_2)$是Negrative Definite, 那么(0,0)是unstable. <br/>

Lyapunov函数的构造

常用的Lyapunov函数:
<br/>
1. V非负
    - 12
2. There we go
3. Now this


In [27]:
V = a*x**2+b*y**2+2*c*x*y
DV = Matrix([[V.diff(x)],[V.diff(y)]])
f = Matrix([[y],[x**2-2*x]])
simplify(transpose(DV)*f)

[2⋅x⋅(x - 2)⋅(b⋅y + c⋅x) + 2⋅y⋅(a⋅x + c⋅y)]

In [28]:
expand(transpose(DV)*f)

⎡               2                    3        2        2⎤
⎣2⋅a⋅x⋅y + 2⋅b⋅x ⋅y - 4⋅b⋅x⋅y + 2⋅c⋅x  - 4⋅c⋅x  + 2⋅c⋅y ⎦

In [88]:
A = Matrix([[-1,-2],[3,4]])

In [91]:
X = exp(A*t)
dv1 = simplify((X**(-1)*Matrix([[2*exp(-t)],[exp(-t)]]))[0])
dv2 = simplify((X**(-1)*Matrix([[2*exp(-t)],[exp(-t)]]))[1])
v1 = integrate(dv1,(t,t0,t))
v2 = integrate(dv2,(t,t0,t))
dv1
dv2
v1
v2

  ⎛   t    ⎞  -3⋅t
2⋅⎝4⋅ℯ  - 3⎠⋅ℯ    

⎛       t⎞  -3⋅t
⎝9 - 8⋅ℯ ⎠⋅ℯ    

     -3      -2      -2⋅t      -3⋅t
- 2⋅ℯ   + 4⋅ℯ   - 4⋅ℯ     + 2⋅ℯ    

     -2      -3      -2⋅t      -3⋅t
- 4⋅ℯ   + 3⋅ℯ   + 4⋅ℯ     - 3⋅ℯ    

In [1]:
pandoc jorek_installation.md -o jorek_installation.pdf

SyntaxError: invalid syntax (Temp/ipykernel_10704/4135991625.py, line 1)